In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import mplleaflet

from geopy.distance import geodesic

import os

import math

from k_medoids_rf import KMedoidsWeighted

%matplotlib inline

In [2]:
data_dir = '../data/'
mamografo_dataset_name = 'RF- Mamógrafos.csv'
use_cols = ['COD_MUN','2018/Dez'] # obtendo os dados mais recentes
municipios_dataset_name = 'dados_mun.csv'
tqdm.pandas()

In [3]:
def dist_to_near_mamografo(row):
    '''
    Retorna a menor distancia a um mamografo, dada a latitude e longitude de uma cidade
    '''
    distancias = cities_mamografo[cities_mamografo.UF == row.UF].apply(lambda x: geodesic((x['LATITUDE'], x['LONGITUDE']), (row.LATITUDE, row.LONGITUDE)).km , axis=1)
    index_min = distancias.idxmin()
    near_city = cities_mamografo.iloc[index_min]
    distancia = geodesic((near_city.LATITUDE, near_city.LONGITUDE), (row.LATITUDE, row.LONGITUDE)).km
    return near_city.COD_MUN, distancia
def population_around_mamografo(codigo_mun):
    '''
    Retorna a soma das populacoes das cidades ao redor de um mamografo
    '''
    cidades_proximas = municipios[municipios.COD_PROX == codigo_mun]
    return cidades_proximas.POPULACAO.astype(int).sum()

In [4]:
# Lendo os dataframes:

mamografo = pd.read_csv(os.path.join(data_dir, mamografo_dataset_name), encoding='latin', index_col=False, 
                     error_bad_lines=False, low_memory=False, usecols=use_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_dataset_name), encoding='latin', index_col=0, 
                     error_bad_lines=False, low_memory=False)

municipios['COD_MUN'] = municipios['COD_MUN'].apply(lambda x: (int) ((x - x%10)/10))
cities_mamografo = pd.merge(mamografo[mamografo['2018/Dez'] > 0], municipios, on='COD_MUN', how='left')
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[1], axis=1), COD_PROX = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[0], axis=1))
estados = municipios.UF.unique()
distancia_max = 70
populacao_max = 240000

In [11]:
UF_analisado = 'CE'
cidades_no_estado = municipios[municipios.UF == UF_analisado]
proximos, distantes = cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max]
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.progress_apply(lambda cod: population_around_mamografo(cod)))
pop_ok, pop_n_ok = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]
mamografo_no_estado

,COD_MUN,2018/Dez,UF,NOME,POPULACAO,LATITUDE,LONGITUDE,CAPITAL,CODIGO_UF,POP_ATENDIDA
103,230110,1,CE,Aracati,72727,-4.55826,-37.7679,0,23,115752
104,230190,3,CE,Barbalha,58855,-7.29820,-39.3021,0,23,85927
105,230250,2,CE,Brejo Santo,48056,-7.48469,-38.9799,0,23,187793
106,230260,1,CE,Camocim,62473,-2.90050,-40.8544,0,23,258837
107,230270,1,CE,Campos Sales,27123,-7.06761,-40.3687,0,23,161777
108,230350,1,CE,Cascavel,70047,-4.12967,-38.2412,0,23,186879
109,230370,2,CE,Caucaia,353932,-3.72797,-38.6619,0,23,353932
110,230410,3,CE,CrateÃºs,74271,-5.16768,-40.6536,0,23,260947
111,230420,3,CE,Crato,128680,-7.21530,-39.4103,0,23,187534
112,230440,18,CE,Fortaleza,2591188,-3.71664,-38.5423,1,23,2591188


In [12]:
plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='yellowgreen', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='darkolivegreen', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.show()

In [13]:
total_cidades, total_populacao, total_mamografos, ideal_mamografos,suficiente,cidades_proximas, cidades_distantes, centros_suficiente, centros_insuficiente = [],[],[],[],[],[],[],[],[]
for estado in estados:
    UF_analisado = estado
    cidades_no_estado = municipios[municipios.UF == estado]
    mamografo_no_estado = cities_mamografo[cities_mamografo.UF == estado]
    mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.apply(lambda cod: population_around_mamografo(cod)))
    
    distantes, proximas = cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max]
    suficientes, insuficientes = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]
    total_populacao_estado = cidades_no_estado.POPULACAO.astype(int).sum()
    total_mamografo_estado = mamografo_no_estado['2018/Dez'].astype(int).sum()
    ideal_mamografo_estado = math.ceil(total_populacao_estado/populacao_max)
    
    total_cidades.append(cidades_no_estado.COD_MUN.count())
    total_populacao.append(total_populacao_estado)
    total_mamografos.append(total_mamografo_estado)
    ideal_mamografos.append(ideal_mamografo_estado)
    suficiente.append(total_mamografo_estado > ideal_mamografo_estado)
    cidades_proximas.append(proximas.COD_MUN.count())
    cidades_distantes.append(distantes.COD_MUN.count())
    centros_suficiente.append(suficientes.COD_MUN.count())
    centros_insuficiente.append(insuficientes.COD_MUN.count())
    
panorama_distribuicao_mamografos_uf = pd.DataFrame({
    'UF': estados,
    'TOTAL_CIDADES': total_cidades,
    'TOTAL_POPULACAO': total_populacao,
    'TOTAL_MAMOGRAGOS': total_mamografos,
    'IDEAL_MAMOGRAFOS': ideal_mamografos,
    'SUFICIENTE': suficiente,
    'CIDADES_PROXIMAS': cidades_proximas,
    'CIDADES_DISTANTES': cidades_distantes,
    'CENTROS_SUFICIENTES': centros_suficiente,
    'CENTROS_INSUFICIENTES': centros_insuficiente
},columns= ['UF', 'TOTAL_MAMOGRAGOS', 'TOTAL_POPULACAO', 'IDEAL_MAMOGRAFOS','SUFICIENTE','TOTAL_CIDADES', 'CIDADES_DISTANTES' , 'CIDADES_PROXIMAS', 'CENTROS_SUFICIENTES', 'CENTROS_INSUFICIENTES'])
panorama_distribuicao_mamografos_uf

,UF,TOTAL_MAMOGRAGOS,TOTAL_POPULACAO,IDEAL_MAMOGRAFOS,SUFICIENTE,TOTAL_CIDADES,CIDADES_DISTANTES,CIDADES_PROXIMAS,CENTROS_SUFICIENTES,CENTROS_INSUFICIENTES
0,RO,14,1768204,8,True,52,29,23,2,3
1,AC,3,803513,4,False,22,14,8,1,1
2,AM,55,3938336,17,True,62,15,47,35,1
3,RR,4,505665,3,True,15,12,3,0,1
4,PA,50,8206923,35,True,144,44,100,8,15
5,AP,2,766679,4,False,16,13,3,0,1
6,TO,18,1515126,7,True,139,80,59,5,2
7,MA,30,6904241,29,True,217,92,125,4,11
8,PI,40,3204028,14,True,224,81,143,9,4
9,CE,69,8904459,38,True,184,14,170,12,13


In [15]:
# panorama_distribuicao_mamografos_uf.CIDADES_DISTANTES / panorama_distribuicao_mamografos_uf.TOTAL_CIDADES
# aplicar o k-medoids para definir novos pontos
UF_analisado = 'CE'
cidades_no_estado = municipios[municipios.UF == UF_analisado]
cidades_com_mamografo = cities_mamografo[cities_mamografo.UF == UF_analisado]['COD_MUN'].values.tolist()
X = cidades_no_estado[['LATITUDE', 'LONGITUDE', 'COD_MUN']].values.tolist()
medoids_fix_idxs = []
for i in range(len(X)):
    cod_mun = X[i][2]
    if cod_mun in cidades_com_mamografo:
        medoids_fix_idxs.append(i)

# quantidade inicial de cidades com mamografos
n_centros = len(medoids_fix_idxs)

# quantidade de novas cidades com novo centro
n_novos = 1
clustering = KMedoidsWeighted(n_clusters=n_centros+n_novos)
medoids_idxs, clusters = clustering.fit(X, medoids_fix_idxs, max_itr=10, verbose=1)

Iteration: 0 ... loss: 55.59189671104862
Iteration: 1 ... loss: 52.21817772483767


In [16]:
# adicionando cidades sugeridas na lista de cidades com mamografos
novos_medoids = np.setdiff1d(medoids_idxs, medoids_fix_idxs)
for i in novos_medoids:
    cod_mun = int(X[i][2])
    nova_cidade = municipios[municipios.COD_MUN == cod_mun].values.tolist()[0]
    cities_mamografo = cities_mamografo.append({'COD_MUN' : cod_mun,
                                                '2018/Dez': 0,
                                                'UF': nova_cidade[0],
                                                'NOME': nova_cidade[2],
                                                'POPULACAO': nova_cidade[3],
                                                'LATITUDE': nova_cidade[4],
                                                'LONGITUDE': nova_cidade[5],
                                                'CAPITAL': nova_cidade[6],
                                                'CODIGO_UF': nova_cidade[7]
                                               } , ignore_index=True)

In [17]:
# atualizando distancias no municipio modificado
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: (dist_to_near_mamografo(row)[1] if (row.UF == UF_analisado) else row.DISTANCIAS), axis=1), COD_PROX = municipios.progress_apply(lambda row: (dist_to_near_mamografo(row)[0] if (row.UF == UF_analisado) else row.COD_PROX), axis=1))
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.apply(lambda cod: population_around_mamografo(cod)))
mamografo_no_estado = mamografo_no_estado.assign(IDEAL_MAMOGRAFO = mamografo_no_estado.POP_ATENDIDA.apply(lambda pop: math.ceil(pop/populacao_max)))


In [90]:
mamografo_no_estado_copy = mamografo_no_estado

In [91]:
# redistribuindo os mamografos

mamografo_no_estado_copy = mamografo_no_estado_copy.sort_values(by=['POP_ATENDIDA'], ascending=False)
total_mamografos = mamografo_no_estado_copy['2018/Dez'].astype(int).sum()
total_cities = mamografo_no_estado_copy.COD_MUN.astype(int).count()
total_pop_atendida = mamografo_no_estado_copy.POP_ATENDIDA.astype(int).sum()
cities_matrix = mamografo_no_estado_copy[['COD_MUN', 'POP_ATENDIDA', 'IDEAL_MAMOGRAFO']].values.tolist()
for i in range(total_cities):
    cities_matrix[i].append(0)

In [92]:
# redistribuir mamografos se o estado for suficiente de mamografos
if (total_pop_atendida / total_mamografos) < populacao_max:
    aux = total_mamografos
    i = 0
    saltos_seguidos = 0
    while (aux > 0) and (saltos_seguidos <= total_cities):
        if (cities_matrix[i][3] >= cities_matrix[i][2]):
            saltos_seguidos = saltos_seguidos + 1
            i = (i + 1)%total_cities
            continue
        saltos_seguidos = 0
        cities_matrix[i][3] = cities_matrix[i][3] + 1
        i = (i + 1)%total_cities
        aux = aux - 1
    i = 0
    while aux > 0:
        cities_matrix[i][3] = cities_matrix[i][3] + 1
        i = (i + 1)%total_cities
        aux = aux - 1 
    

In [93]:
cities_matrix = sorted(cities_matrix, key = lambda k: k[0])
redistirbuicao = []
for city in cities_matrix:
    redistirbuicao.append(city[-1])

In [94]:
mamografo_no_estado_copy = mamografo_no_estado_copy.sort_values(by='COD_MUN')

In [95]:
mamografo_no_estado_copy['REDISTRIBUICAO'] = redistirbuicao

In [96]:
mamografo_no_estado_copy

,COD_MUN,2018/Dez,UF,NOME,POPULACAO,LATITUDE,LONGITUDE,CAPITAL,CODIGO_UF,POP_ATENDIDA,IDEAL_MAMOGRAFO,REDISTRIBUICAO
103,230110,1,CE,Aracati,72727,-4.55826,-37.7679,0,23,115752,1,1
104,230190,3,CE,Barbalha,58855,-7.29820,-39.3021,0,23,85927,1,1
945,230210,0,CE,BaturitÃ©,34949,-4.32598,-38.8812,0,23,350506,2,3
105,230250,2,CE,Brejo Santo,48056,-7.48469,-38.9799,0,23,187793,1,2
106,230260,1,CE,Camocim,62473,-2.90050,-40.8544,0,23,258837,2,3
107,230270,1,CE,Campos Sales,27123,-7.06761,-40.3687,0,23,161777,1,2
108,230350,1,CE,Cascavel,70047,-4.12967,-38.2412,0,23,161756,1,1
109,230370,2,CE,Caucaia,353932,-3.72797,-38.6619,0,23,353932,2,3
110,230410,3,CE,CrateÃºs,74271,-5.16768,-40.6536,0,23,260947,2,3
111,230420,3,CE,Crato,128680,-7.21530,-39.4103,0,23,187534,1,2


In [99]:
proximos, distantes = cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max]
pop_ok, pop_n_ok = mamografo_no_estado_copy[mamografo_no_estado_copy.REDISTRIBUICAO >= mamografo_no_estado_copy.IDEAL_MAMOGRAFO], mamografo_no_estado_copy[mamografo_no_estado_copy.REDISTRIBUICAO < mamografo_no_estado_copy.IDEAL_MAMOGRAFO]

plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='yellowgreen', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='darkolivegreen', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.show()